# Xor with Pytorch: Model class

Here we use a createa a custom Model class (derived from `nn.Model`).

**Note:** Obviously, creating a custom Model is not necesary for such a simple network, this is only for demo/learning pourposes

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from pathlib import Path

Create dataset

In [ ]:
def xor_data_norm(num):
    """ Crete dataset """
    x1 = np.random.randn(num)
    x2 = np.random.randn(num)
    x = np.stack([x1, x2], axis=1).astype('float32')
    y = (2.0 * np.logical_xor(x1 >= 0, x2 >= 0).astype('float32') - 1.0) * 0.9
    return torch.Tensor(x), torch.Tensor(y)

Create model

In [ ]:
# Module class example. It does exactly the same as:
#     nn.Sequential(nn.Linear(...), nn.Tanh(), nn.Linear(...), nn.Tanh())
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer_1 = nn.Linear(2,3)
        self.layer_2 = nn.Linear(3,1)

    def forward(self, x):
        x = self.layer_1(x)
        x = torch.tanh(x)
        x = self.layer_2(x)
        out = torch.tanh(x)
        return out

Train model

In [ ]:
def train(model, x_train, y_train, epochs, lr, momentum):
    device = torch.device("cpu")
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    for n_epoch in range(epochs):
        optimizer.zero_grad()
        output = model(x_train)
        loss = F.mse_loss(output.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        if n_epoch % 50 == 0:
            print(f"Train Epoch: {n_epoch} / {epochs}\tLoss: {loss.item():.6f}")

In [ ]:
x_train, y_train = xor_data_norm(200)
model = Net()
train(model, x_train, y_train, epochs=1000, lr=0.1, momentum=0.8)

In [ ]:
yhat = model(x_train)
for i in range(len(yhat)):
    print(f"{y_train[i]}\t{yhat[i]}\t{x_train[i]}")